In [1]:
import gc
import pandas as pd
import numpy as np
import os

## 데이터 로드

In [3]:
path = "data"

In [4]:
%%time 

aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle':'category'})
departments = pd.read_csv(os.path.join(path, "departments.csv"), dtype={'department_id':np.uint8, 'department': 'category'})
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order':np.uint8,
                                                                                  'reordered': bool})
order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order':np.uint8,
                                                                                  'reordered': bool})
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id':np.uint32,
                                                              'user_id': np.uint32,
                                                              'eval_set': 'category',
                                                              'order_number':np.uint8,
                                                              'order_dow': np.uint8,
                                                              'order_hour_of_day': np.uint8
                                                              })

products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                  'aisle_id': np.uint8,
                                                                  'department_id': np.uint8})

labels = pd.read_pickle('data/previous_products.pkl')

CPU times: user 10.6 s, sys: 704 ms, total: 11.3 s
Wall time: 11.3 s


In [8]:
## orders 데이터를 살펴보면 eval_set이 prior, train, test로 구성된 것을 알 수 있다. 
## 따라서 각각의 데이터를 전처리하기 위해 사전에 나눈다.
orders.ix[[1,74,96]]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,2398795,1,prior,2,3,7,15.0
74,525192,7,train,21,2,11,6.0
96,1376945,11,test,8,6,11,8.0


In [9]:
%%time
orders = orders.loc[(orders.eval_set == 'train') | (orders.eval_set == 'test'), :]

CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 41.2 ms


In [11]:
## prior를 제외하고 train과 test만 뽑았으니 prior가 0으로 표시되는 것을 알 수 있다.
## category로 불러왔기 때문에 데이터가 없더라도 eval_set은 prior에 대한 메타데이터를 갖고 있다.
## 따라서 prior가 0으로 나타나는 것이다. 만약 object로 불러왔다면 prior는 사라질 것이다.

orders.eval_set.value_counts()

train    131209
test      75000
prior         0
Name: eval_set, dtype: int64

In [13]:
# preprocessing#1 에서 처리한 lables 데이터는 user_id가 어떤 product_id를 구매한적 있는지 나타낸다.
# 단 drop_duplicate를 했기 때문에 한 제품을 여러번 구입한 정보는 품고 있지 않다. 단지 label 정보만을 나타낸다.
labels.head(1)

,user_id,product_id
0,202279,33120


In [15]:
orders[['order_id', 'user_id', 'eval_set']].head()

,order_id,user_id,eval_set
10,1187899,1,train
25,1492625,2,train
38,2774568,3,test
44,329954,4,test
49,2196797,5,train


In [16]:
%%time
labels = pd.merge(labels, orders[['order_id', 'user_id', 'eval_set']], on='user_id').drop(['user_id'], axis=1)

CPU times: user 2.16 s, sys: 792 ms, total: 2.95 s
Wall time: 2.95 s


In [17]:
## 결론적으로 어떤 상품이 어떤 order_id와 eval_set에 있는지 labeling한 것이다.
labels.head()

,product_id,order_id,eval_set
0,33120,1050357,train
1,28985,1050357,train
2,9327,1050357,train
3,45918,1050357,train
4,30035,1050357,train


In [18]:
order_train.drop(['add_to_cart_order'], axis=1, inplace=True)

In [19]:
## 이것도 그냥 order_id가 병합해서 여러번 반복되니 order_id를 유니크하게 뽑아낸것
orders = np.unique(labels.order_id)

In [20]:
orders[:10]

array([  1,  17,  34,  36,  38,  96,  98, 112, 137, 170], dtype=uint32)

In [21]:
## 총 206209 건의 주문이 있다. 여기서 중요한 것은 206209가 user_id 수와 같다는것
orders.shape[0]

206209

In [22]:
## 데이터를 chunking 하고 싶으면 folds를 조정하면 된다. 지금은 필요없으니 1로 둔다.
folds = 1
size = orders.shape[0] // folds

In [27]:
## 코드 분해를 위해 fold 강제 할당
fold = 0

In [29]:
## 0 * 206209 = 0
fold * size

0

In [30]:
## 1 * 206209 = 206209
(fold + 1) * size

206209

In [31]:
## 정말 그 자체로 chunk 코드이다.
## orders[fold * size:(fold + 1) * size] --> orders[0:206209] 랑 같다. 즉 전부 갖고와라 가 됨
current = orders[fold * size:(fold + 1) * size]
current

array([      1,      17,      34, ..., 3421058, 3421063, 3421070], dtype=uint32)

In [39]:
labels.head(1)

,product_id,order_id,eval_set
0,33120,1050357,train


In [34]:
labels.order_id[:2]

0    1050357
1    1050357
Name: order_id, dtype: uint32

In [36]:
current[:2]

array([ 1, 17], dtype=uint32)

In [37]:
np.in1d(labels.order_id, current)

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [41]:
## numpy.in1d 는 있냐 없냐를 bool로 판단해주는 함수
## 즉 labels.order_id 에 current가 있으면 True 없으면 False임
## 결론 : current에 있는 order_id를 포함하는 labels만 갖고오겠다는 뜻.
## 청크의 일종이다. 청킹할 경우 뽑아낸 order_id가 current에 저장돼고 current의 labels만 갖고오기 위함.
current = labels.loc[np.in1d(labels.order_id, current), :]

In [42]:
current = pd.merge(order_train, current, on=['order_id', 'product_id'], how='right')

In [44]:
current.head(1)

,order_id,product_id,reordered,eval_set
0,1,49302,True,train


In [45]:
## 앞전에 prior를 제외한 train과 test를 뽑아왔다. 하지만 test의 reordered는 NaN이기 때문에
## 0으로 바꿔준다(False == 0/ True == 1)
current.reordered.fillna(False, inplace=True)

**이 코드는 chunk를 하느냐 마느냐 뿐만아니라, 앞전의 labels가 user_id와	product_id만 있었던 labels를 reordered정보와 eval_set정보도 포함시켜주는 코드이다. 즉 우리가 궁굼한 정보가 reordered 여부이기 때문에 이를 마킹해 준 것이다.**